# **NETTOYAGE DE LA BASE DE DONNÉE DES FOURNITURES**

In [1]:
# Import des libraires necessaires
import pandas as pd
import numpy as np
import os
from pathlib import Path

## 1. CHARGEMENT DES DONNÉES

In [2]:
# Configuration des chemins
BASE_DIR = Path('..')
RAW_DIR = BASE_DIR / 'data' / 'raw' / 'Article'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed' / 'final'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# Charger les données de Yaoundé
input_file = RAW_DIR / 'fourniture_yaoundé.csv'
input_file_1 = RAW_DIR / 'fourniture_douala.csv'

df_yaoundé = pd.read_csv(input_file, sep=";")
print(f"✅ Yaoundé: {len(df_yaoundé)} articles chargés")
print(f"Colonnes: {list(df_yaoundé.columns)}")
print("\nAperçu Yaoundé:")
df_yaoundé.head()

✅ Yaoundé: 187 articles chargés
Colonnes: ['DESIGNATIONS', 'PRIX']

Aperçu Yaoundé:


,DESIGNATIONS,PRIX
0,Cahiers de dessin 16 pages petit format,300
1,Cahiers de dessin 32 pages petit format,500
2,Cahiers de dessin 32 pages grand format,1000
3,Paquet de cahier de dessin petit format,2 000
4,Paquet de cahier de dessin grand format,3 500


In [4]:
# Charger les données de Douala
df_douala = pd.read_csv(input_file_1, sep=";")
print(f"\n✅ Douala: {len(df_douala)} articles chargés")
print(f"Colonnes: {list(df_douala.columns)}")
print("\nAperçu Douala:")
df_douala.head()


✅ Douala: 61 articles chargés
Colonnes: ['DESIGNATIONS', 'PRIX']

Aperçu Douala:


,DESIGNATIONS,PRIX
0,Couvertures cahiers,100
1,Couvertures cahiers transparentes,100
2,Couvre-livres Adhésif rouleau (A),1500
3,Couvre-livres Adhésif rouleau (B),2000
4,Crayon BIC HB,1 500


## 2. NETTOYAGE ET STANDARDISATION

In [5]:
def nettoyer_designation(designation):
    """Nettoie et standardise les désignations"""
    if pd.isna(designation):
        return "Article sans nom"
    
    # Convertir en string et nettoyer
    designation = str(designation).strip()
    
    # Supprimer les caractères spéciaux en début/fin
    designation = designation.strip(".,;-_")
    
    # Remplacer les espaces multiples par un seul
    designation = ' '.join(designation.split())
    
    # Capitaliser la première lettre
    if designation:
        designation = designation[0].upper() + designation[1:]
    
    return designation

def nettoyer_prix(prix):
    """Convertit le prix en entier (francs CFA)"""
    if pd.isna(prix):
        return 0
    
    # Convertir en string d'abord
    prix_str = str(prix).strip()
    
    # Supprimer les espaces et caractères non numériques sauf les chiffres
    prix_str = ''.join(c for c in prix_str if c.isdigit())
    
    try:
        return int(prix_str) if prix_str else 0
    except (ValueError, TypeError):
        return 0

def generer_code_article(designation, ville, index):
    """Génère un code unique pour chaque article"""
    # Prendre les 3 premiers caractères de la désignation
    code_base = ''.join(c for c in designation[:3] if c.isalpha()).upper()
    if len(code_base) < 2:
        code_base = "ART"
    
    # Ajouter la ville et l'index
    ville_code = ville[0].upper()  # Y pour Yaoundé, D pour Douala
    return f"F{code_base}{ville_code}{index:03d}"

In [6]:
# Nettoyer Yaoundé
df_yaoundé_clean = df_yaoundé.copy()
df_yaoundé_clean['DESIGNATIONS'] = df_yaoundé_clean['DESIGNATIONS'].apply(nettoyer_designation)
df_yaoundé_clean['PRIX'] = df_yaoundé_clean['PRIX'].apply(nettoyer_prix)
df_yaoundé_clean['ville'] = 'Yaoundé'

print(f"Avant nettoyage: {len(df_yaoundé)} articles")
print(f"Après nettoyage: {len(df_yaoundé_clean)} articles")
print(f"Prix nuls supprimés: {len(df_yaoundé_clean[df_yaoundé_clean['PRIX'] == 0])}")

Avant nettoyage: 187 articles
Après nettoyage: 187 articles
Prix nuls supprimés: 0


In [7]:
# Nettoyer Douala
df_douala_clean = df_douala.copy()
df_douala_clean['DESIGNATIONS'] = df_douala_clean['DESIGNATIONS'].apply(nettoyer_designation)
df_douala_clean['PRIX'] = df_douala_clean['PRIX'].apply(nettoyer_prix)
df_douala_clean['ville'] = 'Douala'

print(f"Avant nettoyage: {len(df_douala)} articles")
print(f"Après nettoyage: {len(df_douala_clean)} articles")
print(f"Prix nuls supprimés: {len(df_douala_clean[df_douala_clean['PRIX'] == 0])}")

Avant nettoyage: 61 articles
Après nettoyage: 61 articles
Prix nuls supprimés: 0


In [8]:
# Afficher quelques exemples de nettoyage
print("\n📋 Exemples de données nettoyées:")
print("\nYaoundé (5 premiers):")
df_yaoundé_clean[['DESIGNATIONS', 'PRIX', 'ville']].head()


📋 Exemples de données nettoyées:

Yaoundé (5 premiers):


,DESIGNATIONS,PRIX,ville
0,Cahiers de dessin 16 pages petit format,300,Yaoundé
1,Cahiers de dessin 32 pages petit format,500,Yaoundé
2,Cahiers de dessin 32 pages grand format,1000,Yaoundé
3,Paquet de cahier de dessin petit format,2000,Yaoundé
4,Paquet de cahier de dessin grand format,3500,Yaoundé


In [9]:
print("\nDouala (5 premiers):")
df_douala_clean[['DESIGNATIONS', 'PRIX', 'ville']].head()


Douala (5 premiers):


,DESIGNATIONS,PRIX,ville
0,Couvertures cahiers,100,Douala
1,Couvertures cahiers transparentes,100,Douala
2,Couvre-livres Adhésif rouleau (A),1500,Douala
3,Couvre-livres Adhésif rouleau (B),2000,Douala
4,Crayon BIC HB,1500,Douala


## 3. STANDARDISATION DES COLONNES

In [10]:
# Renommer les colonnes pour correspondre au schéma de base
df_yaoundé_clean = df_yaoundé_clean.rename(columns={
    'DESIGNATIONS': 'designation',
    'PRIX': 'prix'
})

df_douala_clean = df_douala_clean.rename(columns={
    'DESIGNATIONS': 'designation', 
    'PRIX': 'prix'
})

print("✅ Colonnes standardisées")

✅ Colonnes standardisées


## 4. GÉNÉRATION DES CODES ARTICLES

In [11]:
# Fusionner temporairement pour créer une liste unique d'articles
df_combined = pd.concat([
    df_yaoundé_clean[['designation']].assign(ville='Yaoundé'),
    df_douala_clean[['designation']].assign(ville='Douala')
], ignore_index=True)

# Identifier les articles uniques par désignation
articles_uniques = df_combined['designation'].unique()
print(f"Articles uniques identifiés: {len(articles_uniques)}")

# Créer un mapping designation -> code
code_mapping = {}
for i, designation in enumerate(articles_uniques, 1):
    code_mapping[designation] = f"FOUR{i:04d}"

print(f"✅ {len(code_mapping)} codes générés")

Articles uniques identifiés: 244
✅ 244 codes générés


In [12]:
# Appliquer les codes aux dataframes
df_yaoundé_clean['code'] = df_yaoundé_clean['designation'].map(code_mapping)
df_douala_clean['code'] = df_douala_clean['designation'].map(code_mapping)

print("\n📋 Aperçu avec codes:")
print("\nYaoundé:")
df_yaoundé_clean[['code', 'designation', 'prix', 'ville']].head()

print("\nDouala:")
df_douala_clean[['code', 'designation', 'prix', 'ville']].head()


📋 Aperçu avec codes:

Yaoundé:

Douala:


,code,designation,prix,ville
0,FOUR0185,Couvertures cahiers,100,Douala
1,FOUR0186,Couvertures cahiers transparentes,100,Douala
2,FOUR0187,Couvre-livres Adhésif rouleau (A),1500,Douala
3,FOUR0188,Couvre-livres Adhésif rouleau (B),2000,Douala
4,FOUR0189,Crayon BIC HB,1500,Douala


## 5. CRÉATION DES TABLES FINALES

In [13]:
# Table articles_fournitures (articles uniques)
articles_fournitures = []
for designation, code in code_mapping.items():
    # Prendre le premier prix trouvé (ou faire une moyenne)
    prix_yaoundé = df_yaoundé_clean[df_yaoundé_clean['designation'] == designation]['prix'].iloc[0] \
                   if len(df_yaoundé_clean[df_yaoundé_clean['designation'] == designation]) > 0 else 0
    
    articles_fournitures.append({
        'code': code,
        'designation': designation,
        'prix': prix_yaoundé,  # Prix de base (Yaoundé)
        'type_article': 'fourniture'
    })

df_articles_fournitures = pd.DataFrame(articles_fournitures)
print(f"✅ Table articles_fournitures: {len(df_articles_fournitures)} articles")

✅ Table articles_fournitures: 244 articles


In [14]:
# Table prix_fournitures_ville (prix par ville)
prix_par_ville = []

# Ajouter les prix de Yaoundé
for _, row in df_yaoundé_clean.iterrows():
    prix_par_ville.append({
        'code': row['code'],
        'ville': 'Yaoundé',
        'prix': row['prix']
    })

# Ajouter les prix de Douala
for _, row in df_douala_clean.iterrows():
    prix_par_ville.append({
        'code': row['code'],
        'ville': 'Douala', 
        'prix': row['prix']
    })

df_prix_ville = pd.DataFrame(prix_par_ville)
print(f"✅ Table prix_fournitures_ville: {len(df_prix_ville)} entrées")

✅ Table prix_fournitures_ville: 248 entrées


## 6. VÉRIFICATIONS ET STATISTIQUES

In [15]:
print("\n📈 Statistiques finales:")
print(f"\n• Articles uniques: {len(df_articles_fournitures)}")
print(f"\n• Prix par ville: {len(df_prix_ville)}")
print(f"\n• Villes: {df_prix_ville['ville'].unique()}")


📈 Statistiques finales:

• Articles uniques: 244

• Prix par ville: 248

• Villes: ['Yaoundé' 'Douala']


In [16]:
# Vérifier les doublons
doublons_articles = df_articles_fournitures['code'].duplicated().sum()
doublons_prix = df_prix_ville[['code', 'ville']].duplicated().sum()
print(f"• Doublons articles: {doublons_articles}")
print(f"• Doublons prix-ville: {doublons_prix}")

• Doublons articles: 0
• Doublons prix-ville: 3


## 7. SAUVEGARDE DES FICHIERS FINAUX

In [17]:
# Sauvegarder les articles fournitures
df_articles_fournitures.to_csv(PROCESSED_DIR / "fournitures_final.csv", index=False)
print(f"✅ Fichier sauvegardé: {PROCESSED_DIR / 'fournitures_final.csv'}")

# Sauvegarde des prix par ville
df_prix_ville.to_csv(PROCESSED_DIR / "prix_fournitures_ville_final.csv", index=False)
print(f"✅ Fichier sauvegardé: {PROCESSED_DIR / 'prix_fournitures_ville_final.csv'}")

✅ Fichier sauvegardé: ../data/processed/final/fournitures_final.csv
✅ Fichier sauvegardé: ../data/processed/final/prix_fournitures_ville_final.csv


In [18]:
# Afficher un aperçu final
print("\nArticles fournitures:")
df_articles_fournitures.head(10)


Articles fournitures:


,code,designation,prix,type_article
0,FOUR0001,Cahiers de dessin 16 pages petit format,300,fourniture
1,FOUR0002,Cahiers de dessin 32 pages petit format,500,fourniture
2,FOUR0003,Cahiers de dessin 32 pages grand format,1000,fourniture
3,FOUR0004,Paquet de cahier de dessin petit format,2000,fourniture
4,FOUR0005,Paquet de cahier de dessin grand format,3500,fourniture
5,FOUR0006,Cahiers double ligne banane,200,fourniture
6,FOUR0007,Cahiers double ligne 32 pages,200,fourniture
7,FOUR0008,Paquet cahiers double ligne banane,4000,fourniture
8,FOUR0009,Paquet de cahiers double ligne 32 pages,2000,fourniture
9,FOUR0010,Cahiers de 50 pages,250,fourniture


In [19]:
print("\nPrix par ville:")
df_prix_ville.head(10)


Prix par ville:


,code,ville,prix
0,FOUR0001,Yaoundé,300
1,FOUR0002,Yaoundé,500
2,FOUR0003,Yaoundé,1000
3,FOUR0004,Yaoundé,2000
4,FOUR0005,Yaoundé,3500
5,FOUR0006,Yaoundé,200
6,FOUR0007,Yaoundé,200
7,FOUR0008,Yaoundé,4000
8,FOUR0009,Yaoundé,2000
9,FOUR0010,Yaoundé,250
